# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")


To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921698494174                   -0.69    5.8         
  2   -7.926168399828       -2.35       -1.22    1.0    340ms
  3   -7.926838239085       -3.17       -2.37    1.8    296ms
  4   -7.926861494464       -4.63       -3.05    2.9    331ms
  5   -7.926861653626       -6.80       -3.45    1.8    324ms
  6   -7.926861672768       -7.72       -3.88    1.9    301ms
  7   -7.926861680075       -8.14       -4.25    1.4    279ms
  8   -7.926861681715       -8.79       -4.79    1.5    284ms
  9   -7.926861681854       -9.86       -5.17    2.0    329ms
 10   -7.926861681870      -10.79       -5.72    1.5    280ms
 11   -7.926861681872      -11.63       -6.24    1.8    290ms
 12   -7.926861681873      -12.69       -7.89    1.9    303ms
 13   -7.926861681873      -14.01       -8.03    5.0    429ms


If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),

which directly yields an AbstractSystem.

In [5]:
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921688092138                   -0.69    5.9         
  2   -7.926159455688       -2.35       -1.22    1.0    272ms
  3   -7.926835222588       -3.17       -2.37    1.5    338ms
  4   -7.926861511015       -4.58       -3.02    2.9    322ms
  5   -7.926861640002       -6.89       -3.39    1.9    288ms
  6   -7.926861668680       -7.54       -3.77    1.8    293ms
  7   -7.926861680403       -7.93       -4.34    1.2    315ms
  8   -7.926861681783       -8.86       -4.89    2.0    301ms
  9   -7.926861681856      -10.13       -5.21    1.6    283ms
 10   -7.926861681871      -10.83       -5.85    1.8    282ms
 11   -7.926861681873      -11.81       -6.96    2.1    321ms
 12   -7.926861681873      -13.42       -7.65    3.2    347ms
 13   -7.926861681873   +  -14.45       -8.06    2.4    308ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [6]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921715304978                   -0.69    5.9         
  2   -7.926164090297       -2.35       -1.22    1.0    274ms
  3   -7.926839161803       -3.17       -2.37    1.8    366ms
  4   -7.926864885467       -4.59       -3.00    2.9    359ms
  5   -7.926865035202       -6.82       -3.31    1.8    312ms
  6   -7.926865076362       -7.39       -3.71    1.6    302ms
  7   -7.926865091850       -7.81       -4.43    1.4    298ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [7]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [8]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")
